In [44]:
from zipfile import ZipFile
zip = ZipFile('Resources/archive.zip')
zip.extractall('Resources')

In [45]:
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [46]:
#Display all the columns (to see which to drop)
pd.set_option("display.max_columns", None)

# Read in Mutual Fund informaiton csv


In [47]:
#Read in CSV data 
#MutualFunds
mutualFunds= pd.read_csv(
    Path("Resources/MutualFunds.csv")
,index_col="fund_symbol")
mutualFunds.head()

,quote_type,region,fund_short_name,fund_long_name,currency,initial_investment,subsequent_investment,fund_category,fund_family,exchange_code,exchange_name,exchange_timezone,management_name,management_bio,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,investment_strategy,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,last_cap_gain,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,fund_max_front_end_sales_load,category_max_front_end_sales_load,fund_max_deferred_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,fund_bond_maturity,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,fund_return_2008,category_return_2008,fund_return_2007,category_return_2007,fund_return_2006,category_return_2006,fund_return_2005,category_return_2005,fund_return_2004,category_return_2004,fund_return_2003,category_return_2003,fund_return_2002,category_return_2002,fund_return_2001,category_return_2001,fund_return_2000,category_return_2000,quarters_up,quarters_down,fund_return_2021_q3,fund_return_2021_q2,fund_return_2021_q1,fund_return_2020_q4,fund_return_2020_q3,fund_return_2020_q2,fund_return_2020_q1,fund_return_2019_q4,fund_return_2019_q3,fund_return_2019_q2,fund_return_2019_q1,fund_return_2018_q4,fund_return_2018_q3,fund_return_2018_q2,fund_return_2018_q1,fund_return_2017_q4,fund_return_2017_q3,fund_return_2017_q2,fund_return_2017_q1,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,fund_return_2014_q4,fund_return_2014_q3,fund_return_2014_q2,fund_return_2014_q1,fund_return_2013_q4,fund_return_2013_q3,fund_return_2013_q2,fund_return_2013_q1,fund_return_2

In [48]:
# Find columns with only 1 value to drop
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

quote_type                  1
region                      1
fund_short_name          3044
fund_long_name           6644
currency                    1
fund_category             119
fund_family               310
exchange_code               1
exchange_name               1
exchange_timezone           1
management_name          1388
management_bio           1300
management_start_date    2316
investment_strategy      2245
inception_date           4499
investment_type             3
size_type                   3
top10_holdings           2347
returns_as_of_date         51
esg_peer_group             86
dtype: int64

In [49]:
#convert dates to date
mutualFunds["inception_date"] = pd.to_datetime(mutualFunds["inception_date"])
mutualFunds["management_start_date"] = pd.to_datetime(mutualFunds["management_start_date"])
mutualFunds["returns_as_of_date"] = pd.to_datetime(mutualFunds["returns_as_of_date"])

In [50]:
mutualFunds_counts= mutualFunds.loc[: ,mutualFunds.dtypes=="object"].nunique()
mutualFunds_counts_one= mutualFunds_counts[mutualFunds_counts == 1].index.to_list()
print(mutualFunds_counts_one)

['quote_type', 'region', 'currency', 'exchange_code', 'exchange_name', 'exchange_timezone']


In [51]:
#Drop columns with only 1 unique value
mutualFunds.drop(columns=mutualFunds_counts_one,inplace=True)
mutualFunds

,fund_short_name,fund_long_name,initial_investment,subsequent_investment,fund_category,fund_family,management_name,management_bio,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,investment_strategy,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,last_cap_gain,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,fund_max_front_end_sales_load,category_max_front_end_sales_load,fund_max_deferred_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,fund_bond_maturity,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,fund_return_2008,category_return_2008,fund_return_2007,category_return_2007,fund_return_2006,category_return_2006,fund_return_2005,category_return_2005,fund_return_2004,category_return_2004,fund_return_2003,category_return_2003,fund_return_2002,category_return_2002,fund_return_2001,category_return_2001,fund_return_2000,category_return_2000,quarters_up,quarters_down,fund_return_2021_q3,fund_return_2021_q2,fund_return_2021_q1,fund_return_2020_q4,fund_return_2020_q3,fund_return_2020_q2,fund_return_2020_q1,fund_return_2019_q4,fund_return_2019_q3,fund_return_2019_q2,fund_return_2019_q1,fund_return_2018_q4,fund_return_2018_q3,fund_return_2018_q2,fund_return_2018_q1,fund_return_2017_q4,fund_return_2017_q3,fund_return_2017_q2,fund_return_2017_q1,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,fund_return_2014_q4,fund_return_2014_q3,fund_return_2014_q2,fund_return_2014_q1,fund_return_2013_q4,fund_return_2013_q3,fund_return_2013_q2,fund_return_2013_q1,fund_return_2012_q4,fund_return_2012_q3,fund_return_2012_q2,fund_return_2012_q1,fund_r

_We noticed that there are many `NaN` values throughout the data set. We will now explore which columns/rows have NaN and will remove an appropriate amount of columns/rows._

In [52]:
mutualFunds.dropna(axis = 0)

,fund_short_name,fund_long_name,initial_investment,subsequent_investment,fund_category,fund_family,management_name,management_bio,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,investment_strategy,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,last_cap_gain,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,fund_max_front_end_sales_load,category_max_front_end_sales_load,fund_max_deferred_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,fund_bond_maturity,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,fund_return_2008,category_return_2008,fund_return_2007,category_return_2007,fund_return_2006,category_return_2006,fund_return_2005,category_return_2005,fund_return_2004,category_return_2004,fund_return_2003,category_return_2003,fund_return_2002,category_return_2002,fund_return_2001,category_return_2001,fund_return_2000,category_return_2000,quarters_up,quarters_down,fund_return_2021_q3,fund_return_2021_q2,fund_return_2021_q1,fund_return_2020_q4,fund_return_2020_q3,fund_return_2020_q2,fund_return_2020_q1,fund_return_2019_q4,fund_return_2019_q3,fund_return_2019_q2,fund_return_2019_q1,fund_return_2018_q4,fund_return_2018_q3,fund_return_2018_q2,fund_return_2018_q1,fund_return_2017_q4,fund_return_2017_q3,fund_return_2017_q2,fund_return_2017_q1,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,fund_return_2014_q4,fund_return_2014_q3,fund_return_2014_q2,fund_return_2014_q1,fund_return_2013_q4,fund_return_2013_q3,fund_return_2013_q2,fund_return_2013_q1,fund_return_2012_q4,fund_return_2012_q3,fund_return_2012_q2,fund_return_2012_q1,fund_r

If we drop rows that have an `NaN`, then all data is removed. Instead, let's look at the columns that have missing data.

In [53]:
# Looking at each column and the count of NaN in each column
list = []
for index, row in pd.DataFrame(mutualFunds.isna().sum()).iterrows():
    list.append((index,row))


In [54]:
list[0]

('fund_short_name',
 0    824
 Name: fund_short_name, dtype: int64)

We plan to find the number of columns that have above a certain percentage of `NaN` and then remove those columns. We have chosen 60% so far.

In [55]:
threshold = 0.6*len(mutualFunds)
drop_columns = []
for i in range(len(list)):
    if list[i][1][0] >= threshold:
        drop_columns.append(list[i][0])

In [56]:
print(len(drop_columns))
drop_columns


49


['subsequent_investment',
 'last_cap_gain',
 'fund_max_front_end_sales_load',
 'fund_max_deferred_sales_load',
 'fund_bond_maturity',
 'fund_return_2008',
 'fund_return_2007',
 'fund_return_2006',
 'fund_return_2005',
 'fund_return_2004',
 'fund_return_2003',
 'fund_return_2002',
 'fund_return_2001',
 'fund_return_2000',
 'fund_return_2021_q3',
 'fund_return_2021_q2',
 'fund_return_2008_q4',
 'fund_return_2007_q4',
 'fund_return_2007_q3',
 'fund_return_2007_q2',
 'fund_return_2007_q1',
 'fund_return_2006_q4',
 'fund_return_2006_q3',
 'fund_return_2006_q2',
 'fund_return_2006_q1',
 'fund_return_2005_q4',
 'fund_return_2005_q3',
 'fund_return_2005_q2',
 'fund_return_2005_q1',
 'fund_return_2004_q4',
 'fund_return_2004_q3',
 'fund_return_2004_q2',
 'fund_return_2004_q1',
 'fund_return_2003_q4',
 'fund_return_2003_q3',
 'fund_return_2003_q2',
 'fund_return_2003_q1',
 'fund_return_2002_q4',
 'fund_return_2002_q3',
 'fund_return_2002_q2',
 'fund_return_2002_q1',
 'fund_return_2001_q4',
 'fun

In [57]:
mutualFunds.drop(columns=drop_columns, inplace = True)
mutualFunds

,fund_short_name,fund_long_name,initial_investment,fund_category,fund_family,management_name,management_bio,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,investment_strategy,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,category_max_front_end_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,category_return_2008,category_return_2007,category_return_2006,category_return_2005,category_return_2004,category_return_2003,category_return_2002,category_return_2001,category_return_2000,quarters_up,quarters_down,fund_return_2021_q1,fund_return_2020_q4,fund_return_2020_q3,fund_return_2020_q2,fund_return_2020_q1,fund_return_2019_q4,fund_return_2019_q3,fund_return_2019_q2,fund_return_2019_q1,fund_return_2018_q4,fund_return_2018_q3,fund_return_2018_q2,fund_return_2018_q1,fund_return_2017_q4,fund_return_2017_q3,fund_return_2017_q2,fund_return_2017_q1,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,fund_return_2014_q4,fund_return_2014_q3,fund_return_2014_q2,fund_return_2014_q1,fund_return_2013_q4,fund_return_2013_q3,fund_return_2013_q2,fund_return_2013_q1,fund_return_2012_q4,fund_return_2012_q3,fund_return_2012_q2,fund_return_2012_q1,fund_return_2011_q4,fund_return_2011_q3,fund_return_2011_q2,fund_return_2011_q1,fund_return_2010_q4,fund_return_2010_q3,fund_return_2010_q2,fund_return_2010_q1,fund_return_2009_q4,fund_return_2009_q3,fund_return_2009_q2,fund_return_2009_q1,fund_return_2008_q3,fund_return_2008_q2,fund_return_2008_q1,fund_alpha_3y

In [58]:
# Find columns with only 1 value to drop
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

fund_short_name        3044
fund_long_name         6644
fund_category           119
fund_family             310
management_name        1388
management_bio         1300
investment_strategy    2245
investment_type           3
size_type                 3
top10_holdings         2347
esg_peer_group           86
dtype: int64

In [59]:
#Drop unnecessary columns
mutualFunds.drop(columns=["fund_short_name", "fund_long_name","management_name", "management_bio", "investment_strategy"],inplace=True)


In [60]:
#Remove quarterly data
mutualFunds.drop(columns=["fund_return_2021_q1",
"fund_return_2020_q4","fund_return_2020_q3","fund_return_2020_q2","fund_return_2020_q1",
"fund_return_2019_q4","fund_return_2019_q3","fund_return_2019_q2","fund_return_2019_q1",
"fund_return_2018_q4","fund_return_2018_q3","fund_return_2018_q2","fund_return_2018_q1",
"fund_return_2017_q4","fund_return_2017_q3","fund_return_2017_q2","fund_return_2017_q1",
"fund_return_2016_q4","fund_return_2016_q3","fund_return_2016_q2","fund_return_2016_q1",
"fund_return_2015_q4","fund_return_2015_q3","fund_return_2015_q2","fund_return_2015_q1",
"fund_return_2014_q4","fund_return_2014_q3","fund_return_2014_q2","fund_return_2014_q1",
"fund_return_2013_q4","fund_return_2013_q3","fund_return_2013_q2","fund_return_2013_q1",
"fund_return_2012_q4","fund_return_2012_q3","fund_return_2012_q2","fund_return_2012_q1",
"fund_return_2011_q4","fund_return_2011_q3","fund_return_2011_q2","fund_return_2011_q1",
"fund_return_2010_q4","fund_return_2010_q3","fund_return_2010_q2","fund_return_2010_q1",
"fund_return_2009_q4","fund_return_2009_q3","fund_return_2009_q2","fund_return_2009_q1",
"fund_return_2008_q3","fund_return_2008_q2","fund_return_2008_q1",
"fund_alpha_3years","fund_beta_3years","fund_mean_annual_return_3years","fund_r_squared_3years","fund_stdev_3years","fund_sharpe_ratio_3years","fund_treynor_ratio_3years",
"fund_alpha_5years","fund_beta_5years","fund_mean_annual_return_5years","fund_r_squared_5years","fund_stdev_5years","fund_sharpe_ratio_5years","fund_treynor_ratio_5years",
"fund_alpha_10years","fund_beta_10years","fund_mean_annual_return_10years","fund_r_squared_10years","fund_stdev_10years","fund_sharpe_ratio_10years","fund_treynor_ratio_10years",
"fund_return_category_rank_ytd","fund_return_category_rank_1month","fund_return_category_rank_3months","fund_return_category_rank_1year","fund_return_category_rank_3years",
"fund_return_category_rank_5years","load_adj_return_1year","load_adj_return_3years","load_adj_return_5years","load_adj_return_10years",
"top10_holdings"],inplace=True)

In [61]:
#Remove metrics that we can caluclate ourselves
#keep: "esg_score","environment_score", "sustainability_score", "sustainability_rank",   "social_score",  "governance_score", 
mutualFunds.drop(columns=["esg_peer_count","peer_esg_min", "peer_esg_avg", "peer_esg_max",
"peer_environment_min", "peer_environment_avg", "peer_environment_max", 
"peer_social_min", "peer_social_avg", "peer_social_max",
"peer_governance_min", "peer_governance_avg", "peer_governance_max"],inplace=True)


In [62]:
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

fund_category      119
fund_family        310
investment_type      3
size_type            3
esg_peer_group      86
dtype: int64

Done with Data cleanup and preprocessing


In [63]:
# Calculate the average for each column
column_means = mutualFunds.mean(skipna=True, numeric_only=True)
# Replace NaN values in each column with the respective column average
mutualFunds.fillna(column_means, inplace=True)

In [64]:
mutualFunds.dropna(inplace= True)
mutualFunds

,initial_investment,fund_category,fund_family,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,category_max_front_end_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,category_return_2008,category_return_2007,category_return_2006,category_return_2005,category_return_2004,category_return_2003,category_return_2002,category_return_2001,category_return_2000,quarters_up,quarters_down,sustainability_score,sustainability_rank,esg_peer_group,esg_score,environment_score,social_score,governance_score
fund_symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAAAX,1000.0,World Allocation,DWS,2015-01-15,2.979347e+09,0.21026,12.788,12.369,2.44,0.18683,13.06,-0.51,-0.03905,10.62,1.93,0.18173,0.018600,3.0,3.0,2007-07-30,0.146600,0.7400,Value,Large,0.0122,0.0102,0.0122,0.0136,0.002400,0.0504,0.0136,968.0,1264.0,2105.0,0.0167,0.6436,0.1142,0.2255,0.0000,0.0000,0.1607,0.0088,0.0190,0.0293,0.1852,0.0000,0.0000,0.1413,0.3520,0.0012,0.1025,1.91,2.28,12.09,11.96,23.34,19.86,2.36,1.73,22401.91,68472.90,-1.390000,9.16,6.110000,4.906846,5.800000,0.0,0.340689,0.104272,0.126869,0.168703,0.099992,0.082922,0.026165,0.050383,0.2546,3.0,2021-11-24,0.21026,0.0858,0.00077,0.0172,0.03920,0.0683,0.22970,0.2770,0.138110,0.0784,0.090780,0.0822,0.060580,0.0591,0.08110,0.1349,-0.112300,-0.1313,9.000000,4.000000,0.037030,0.06177,0.214260,0.16097,-0.053690,-0.08259,0.146720,0.14786,0.040080,0.06038,-0.097140,-0.04148,0.030260,0.01536,0.008210,0.10069,0.09

# Binning!

In [65]:
mutualFunds.loc[:,mutualFunds.dtypes == "object"].dtypes

fund_category      object
fund_family        object
investment_type    object
size_type          object
esg_peer_group     object
dtype: object

Binning `fund_category`

In [66]:
fund_category_type_count = mutualFunds["fund_category"].value_counts()[mutualFunds["fund_category"].value_counts() > 150]
print(100*fund_category_type_count.sum()/len(mutualFunds))
print(fund_category_type_count.count())

84.52316076294278
31


In [67]:
for cat in mutualFunds["fund_category"]:
    if cat in fund_category_type_count.index.to_list():
        next
    else:
        mutualFunds["fund_category"] = mutualFunds["fund_category"].replace(cat, "Other")

In [68]:
mutualFunds["fund_category"].value_counts()

fund_category
Other                            2272
Large Blend                      1169
Large Growth                     1153
Large Value                      1054
Diversified Emerging Mkts         702
Foreign Large Blend               653
Allocation--50% to 70% Equity     597
Small Growth                      573
Small Blend                       563
Mid-Cap Growth                    543
Foreign Large Growth              416
Small Value                       392
World Allocation                  386
Mid-Cap Value                     366
Mid-Cap Blend                     329
World Large-Stock Blend           304
Foreign Large Value               298
Allocation--70% to 85% Equity     277
Target-Date 2050                  207
Target-Date 2035                  205
Target-Date 2055                  205
Target-Date 2040                  204
Target-Date 2045                  203
Real Estate                       202
Target-Date 2060+                 201
Target-Date 2030                  19

Binning `fund_family_type_count`

In [69]:
fund_family_type_count = mutualFunds["fund_family"].value_counts()[mutualFunds["fund_family"].value_counts() > 150]
print(100*fund_family_type_count.sum()/len(mutualFunds))
print(fund_family_type_count.count())

64.1757493188011
19


In [70]:
for cat in mutualFunds["fund_family"]:
    if cat in fund_family_type_count.index.to_list():
        next
    else:
        mutualFunds["fund_family"] = mutualFunds["fund_family"].replace(cat, "Other")

In [71]:
mutualFunds["fund_family"].value_counts()

fund_family
Other                              5259
American Century Investments       1128
Fidelity Investments               1090
Columbia Threadneedle              1004
American Funds                      940
Eaton Vance                         613
BlackRock                           607
AllianceBernstein                   561
BNY Mellon                          489
Invesco                             382
MFS                                 366
Delaware Funds by Macquarie         352
Dimensional Fund Advisors           297
Wells Fargo Funds                   283
Franklin Templeton Investments      278
Virtus                              234
American Beacon                     230
Calvert Research and Management     205
Alger                               198
Calamos                             164
Name: count, dtype: int64

Binning `esg_peer_group`

In [72]:
#finding threshhold to bin esg_peer_group
esg_peer_type_count = mutualFunds["esg_peer_group"].value_counts()[mutualFunds["esg_peer_group"].value_counts() > 100]
print(100* esg_peer_type_count.sum()/len(mutualFunds))
print(esg_peer_type_count.count())

88.9441416893733
36


In [73]:
#check the bins of esg_peer_group
esg_peer_type_count.index.to_list()
#replace bins in dataframe for esg_peer_group
for esg in mutualFunds["esg_peer_group"]:
    if esg in esg_peer_type_count.index.to_list():
        next
    else:
        mutualFunds["esg_peer_group"] = mutualFunds["esg_peer_group"].replace(esg, "Other")

In [74]:
mutualFunds["esg_peer_group"].value_counts()

esg_peer_group
Other                                    1623
US Fund Large Growth                     1209
US Fund Large Blend                      1142
US Fund Large Value                      1031
US Fund Diversified Emerging Mkts         697
US Fund Foreign Large Blend               652
US Fund Small Blend                       576
US Fund Allocation--50% to 70% Equity     575
US Fund Small Growth                      567
US Fund Mid-Cap Growth                    551
US Fund Foreign Large Growth              419
US Fund World Allocation                  383
US Fund Small Value                       371
US Fund Mid-Cap Value                     364
US Fund Mid-Cap Blend                     347
US Fund World Large Stock                 314
US Fund Foreign Large Value               298
US Fund Allocation--70% to 85% Equity     295
US Fund Target-Date 2050                  207
US Fund Allocation--30% to 50% Equity     206
US Fund Target-Date 2055                  205
US Fund Target-Date

In [75]:
mutualFunds

,initial_investment,fund_category,fund_family,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,category_max_front_end_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,category_return_2008,category_return_2007,category_return_2006,category_return_2005,category_return_2004,category_return_2003,category_return_2002,category_return_2001,category_return_2000,quarters_up,quarters_down,sustainability_score,sustainability_rank,esg_peer_group,esg_score,environment_score,social_score,governance_score
fund_symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAAAX,1000.0,World Allocation,Other,2015-01-15,2.979347e+09,0.21026,12.788,12.369,2.44,0.18683,13.06,-0.51,-0.03905,10.62,1.93,0.18173,0.018600,3.0,3.0,2007-07-30,0.146600,0.7400,Value,Large,0.0122,0.0102,0.0122,0.0136,0.002400,0.0504,0.0136,968.0,1264.0,2105.0,0.0167,0.6436,0.1142,0.2255,0.0000,0.0000,0.1607,0.0088,0.0190,0.0293,0.1852,0.0000,0.0000,0.1413,0.3520,0.0012,0.1025,1.91,2.28,12.09,11.96,23.34,19.86,2.36,1.73,22401.91,68472.90,-1.390000,9.16,6.110000,4.906846,5.800000,0.0,0.340689,0.104272,0.126869,0.168703,0.099992,0.082922,0.026165,0.050383,0.2546,3.0,2021-11-24,0.21026,0.0858,0.00077,0.0172,0.03920,0.0683,0.22970,0.2770,0.138110,0.0784,0.090780,0.0822,0.060580,0.0591,0.08110,0.1349,-0.112300,-0.1313,9.000000,4.000000,0.037030,0.06177,0.214260,0.16097,-0.053690,-0.08259,0.146720,0.14786,0.040080,0.06038,-0.097140,-0.04148,0.030260,0.01536,0.008210,0.10069,0.

# Read In CSV data for Mutual Fund prices A-Z


In [33]:
#Read in CSV data 
#MutualFund prices A-E
df_AE= pd.read_csv(
    Path("Resources/MutualFund Prices - A-E.csv")
)
df_AE.head()

#set index
df_AE.set_index("fund_symbol", inplace=True)
df_AE["price_date"]=pd.to_datetime(df_AE['price_date'])

In [34]:
#Read in CSV data 
#MutualFund prices F-K
df_FK= pd.read_csv(
    Path("Resources/MutualFund Prices - F-K.csv")
)
df_FK.head()

#set index
df_FK.set_index("fund_symbol", inplace=True)
df_FK["price_date"]=pd.to_datetime(df_FK['price_date'])

In [35]:
#Read in CSV data 
#MutualFund prices L-P
df_LP= pd.read_csv(
    Path("Resources/MutualFund Prices - L-P.csv")
)
df_LP.head()

#set index
df_LP.set_index("fund_symbol", inplace=True)
df_LP["price_date"]=pd.to_datetime(df_LP['price_date'])

In [36]:
#Read in CSV data 
#MutualFund prices Q-Z
df_QZ= pd.read_csv(
    Path("Resources/MutualFund Prices - Q-Z.csv")
)
df_QZ.head()

#set index
df_QZ.set_index("fund_symbol", inplace=True)
df_QZ["price_date"]=pd.to_datetime(df_QZ['price_date'])

In [37]:
# #Concatonate dataframes
# mutual_fund_df= pd.concat([df_AE,df_FK,df_LP,df_QZ])
# mutual_fund_df.shape 

In [38]:
#set index
# mutual_fund_df.set_index("fund_symbol", inplace=True)

In [39]:
#look at the datatypes
# mutual_fund_df.dtypes

In [40]:
#Convert to date
# mutual_fund_df["price_date"]=pd.to_datetime(mutual_fund_df['price_date'])

In [41]:
#check to see if price_date datatype has successfully changed
# mutual_fund_df.dtypes

# Merge mutualFunds and mutual_fund_df on index= "fund_symbol"

In [ ]:
# #merging mutualFunds and mutual_fund_dfs (in chunks)
#merging A-E
merged_df_AE= pd.merge(mutualFunds, df_AE, on= "fund_symbol", how= "inner")
merged_df_AE


In [79]:
import findspark
findspark.init()
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Read in data from S3 Buckets
from pyspark import SparkFiles
file_path = "Resources/MutualFund Prices - A-E.csv"
df_AE = spark.read.csv(SparkFiles.get(file_path), header=True, inferSchema=True)

# Show DataFrame
df_AE.show()


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/private/var/folders/vd/r058js812ld0xgwzbqh1j9c00000gn/T/spark-4df4331e-fc65-44a3-9677-d3c65c1b7eee/userFiles-91862dae-d54e-43b0-bce1-4440db1a16bf/Resources/MutualFund Prices - A-E.csv.

In [ ]:
#merging F-K
merged_df_FK= pd.merge(mutualFunds, df_FK, on= "fund_symbol", how= "inner")
merged_df_FK


In [ ]:
#merging L-P
merged_df_LP= pd.merge(mutualFunds, df_LP, on= "fund_symbol", how= "inner")
merged_df_LP


In [ ]:
#merging Q-Z
merged_df_QZ= pd.merge(mutualFunds, df_LP, on= "fund_symbol", how= "inner")
merged_df_QZ

In [ ]:
# #Concatonate dataframes
mutual_fund_df= pd.concat([merged_df_AE,merged_df_FK,merged_df_LP,merged_df_QZ])
mutual_fund_df.shape 

In [ ]:
from pandasql import sqldf

# Define your SQL query
query = '''
    SELECT *
    FROM mutualFunds
    JOIN mutual_fund_df ON mutualFunds.fund_symbol = mutual_fund_df.index
'''

# Execute the SQL query
merged_df = sqldf(query, locals())

# Display the merged DataFrame
print(merged_df)


# Post merging and binning:

In [ ]:
# Import Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [ ]:
# Convert categorical data to numeric with "pd.get_dummies"
merged_df= pd.get_dummies(merged_df, dtype=float)
merged_df.head()

In [ ]:
# Split our preprocessed data into our features and target array
y= merged_df["target"]
X= merged_df.drop(columns=["target"])

In [ ]:
### Split the preprocessed data into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Creating StandardScaler instance
scaler= StandardScaler()
# Fitting Standard Scaler
x_scaler= scaler.fit(X_train)
# Scaling Data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Building the Model

In [ ]:
### Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

In [ ]:
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

### Compile, Train, Evaluate the Model

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

In [ ]:
# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
